In [1]:
from pathlib import Path
import polars as pl
import duckdb
from datasets import Dataset, concatenate_datasets, load_from_disk
import numpy as np
from autofaiss import build_index
from core_pro.ultilities import make_sync_folder
import sys
sys.path.extend([str(Path.home() / 'PycharmProjects/item_matching')])

# from src.item_matching.build_index.func_img import PipelineImage
# from item_matching.model.model import Model
# from func import draw_images

In [5]:
path = make_sync_folder('Item_Matching_Test')
path_db = path / 'data_sample_ELHA.parquet'

# db
query = f"""
select *
from parquet_scan('{str(path_db)}')
"""
df_db = (
    duckdb.sql(query).pl()
    .select(pl.all().name.prefix('db_'))
)

# q
df_q = df_db.clone()
df_q.columns = [f'q_{i.split('db_')[1]}' for i in df_db.columns]
df_q.head()

q_item_id,q_item_name,q_shop_id,q_shop_name,q_level1_global_be_category,q_level2_global_be_category,q_level3_global_be_category,q_cluster,q_description,q_images
i64,str,i64,str,str,str,str,str,str,str
20084814019,"""Ultra Ốp Điện Thoại pc Silicon…",264148388,"""DESN.VN""","""Mobile & Gadgets""","""Accessories""","""Cases, Covers, & Skins""","""ELHA""","""[{""t"":""Chú ý! Chú ý! Chú ý!\nN…","""cn-11134207-7r98o-lmem658l6u5e…"
24275644657,"""Mowin - Dành cho iPhone 15 Pro…",179790847,"""mowin.vn""","""Mobile & Gadgets""","""Accessories""","""Cases, Covers, & Skins""","""ELHA""","""[{""i"":""sg-11134202-7rd5u-lubux…","""sg-11134201-7rd6h-lubuwsfjov3r…"
25854759539,"""Ốp lưng iphone trau dồi 11 12 …",1024607561,"""kirby0527.vn""","""Mobile & Gadgets""","""Accessories""","""Cases, Covers, & Skins""","""ELHA""","""[{""t"":""💕Chào mừng đến với cửa …","""sg-11134201-7rbmy-lplxvsl3lbcl…"
19257272185,"""[HÀNG LOẠI TỐT] Dây cáp sạc dà…",190253918,"""Phụ Kiện Chấn Thiên""","""Mobile & Gadgets""","""Accessories""","""Cables, Chargers & Converters""","""ELHA""","""[{""t"":""DÂY SẠC HỖ TRỢ SẠC CHO …","""vn-11134207-7r98o-lu5uomvfvdht…"
24872438141,"""Dây Cáp Sạc Nhanh 2 Đầu Type C…",718584929,"""ANBELER""","""Mobile & Gadgets""","""Accessories""","""Cables, Chargers & Converters""","""ELHA""","""[{""t"":""✅ Dây Cáp Sạc Nhanh 2 Đ…","""vn-11134207-7r98o-lwfi09222ok9…"


In [6]:
df_db.columns

['db_item_id',
 'db_item_name',
 'db_shop_id',
 'db_shop_name',
 'db_level1_global_be_category',
 'db_level2_global_be_category',
 'db_level3_global_be_category',
 'db_cluster',
 'db_description',
 'db_images']

## 2) Embeddings

Use datasets and clip to transform images to vectors

In [3]:
model = Model()
model.get_img_model()

INFO | __init__ | [Model] Run on: cuda


In [7]:
dataset = Dataset.from_polars(df_db)
dataset = dataset.map(
    model.process_image,
    batch_size=512,
    batched=True,
    fn_kwargs={'col': 'db_file_path'}
)

dataset.set_format(type='torch', columns=['image_embed'], output_all_columns=True)
dataset = dataset.map(model.pp_normalize, batched=True, fn_kwargs={'col': 'image_embed'})
dataset.set_format(type='numpy', columns=['image_embed'], output_all_columns=True)

# save to disk
path_tmp_array = Path('tmp/array')
path_tmp_ds = Path('tmp/ds')
np.save(path_tmp_array / 'array.npy', dataset['image_embed'])
dataset.save_to_disk(path_tmp_ds / 'ds')

Map:   0%|          | 0/145 [00:00<?, ? examples/s]

Map:   0%|          | 0/145 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/145 [00:00<?, ? examples/s]

## 3) Indexing

Build index to search items

In [8]:
path_index = Path('tmp/index')
build_index(
    str(path_tmp_array),
    index_path=str(path_index / f'ip.index'),
    index_infos_path=str(path_index / f'index.json'),
    save_on_disk=True,
    metric_type='ip',
    verbose=30,
)

  0%|                                                                              | 0/1 [00:00<?, ?it/s]


(<faiss.swigfaiss_avx2.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *' at 0x7c37e5fba670> >,
 {'index_key': 'Flat',
  'index_param': '',
  'index_path': 'tmp/index/ip.index',
  'size in bytes': 445485,
  'avg_search_speed_ms': 0.01423576567293342,
  '99p_search_speed_ms': 0.03772199779632523,
  'reconstruction error %': 0.0,
  'nb vectors': 145,
  'vectors dimension': 768,
  'compression ratio': 0.9998989864978619})

Load index into datasets

In [9]:
dataset_db = concatenate_datasets([
    load_from_disk(str(f)) for f in sorted(path_tmp_ds.glob('*'))
])

# add index
dataset_db.load_faiss_index('img_embed', path_index / f'ip.index')

## 4) Retrieve

Batch search top-k from datasets

In [10]:
score, result = dataset_db.get_nearest_examples_batch(
    'img_embed',
    np.asarray(dataset_db['img_embed']),
    k=5
)

dict_ = {'score_img': [list(i) for i in score]}
df_score = pl.DataFrame(dict_)
df_result = pl.DataFrame(result).drop(['img_embed'])

KeyError: "Column img_embed not in the dataset. Current columns in the dataset: ['db_l0_category', 'db_model_id', 'db_item_name', 'db_model_name', 'db_model_price', 'db_image_show', 'db_item_url', 'db_image_url', 'db_item_name_clean', 'db_file_path', 'db_exists', 'image_embed']"

## 5) Post process

In [8]:
df_match = pl.concat([df_q, df_result, df_score], how='horizontal')
col_explode = [i for i in df_match.columns if 'db' in i] + ['score_img']
df_match = df_match.explode(col_explode)

In [12]:
df_match.to_pandas()

In [17]:
draw_images(df_match, 2999787165)

In [16]:
draw_images(df_match, 3099789245)

In [14]:
draw_images(df_match, 2999838844)

In [15]:
draw_images(df_match, 3099458499)

In [11]:
# df_match.write_csv(path / 'match.csv')